In [8]:
import math
import os
import random
import sys
import time
import re

import editdistance as ed
import numpy as np

Fqu = []
Flo = []
Ffi = []

with open('./rand1.fi') as f_fi:
    with open('./rand1.qu') as f_qu:
        with open('./rand1.lo') as f_lo:
            schema, query, logic = f_fi.readline(), f_qu.readline(), f_lo.readline()
            idx = 0
            while schema and query and logic:
                Fqu.append(query)
                Flo.append(logic)
                Ffi.append(schema)
                schema, query, logic = f_fi.readline(), f_qu.readline(), f_lo.readline()
                
num = np.random.permutation(len(Fqu))
print num
f_fi1 = open('./rand.fi', 'w')
f_lo1 = open('./rand.lo', 'w')
f_qu1 = open('./rand.qu', 'w')

for i in range(len(num)):
    f_lo1.write(Flo[num[i]])
    f_fi1.write(Ffi[num[i]])
    f_qu1.write(Fqu[num[i]])

f_fi1.close()
f_qu1.close()
f_lo1.close()

[ 2099 10415   406 ...,  1761  8490  7321]


In [9]:
f_fi1 = open('../data/rand.fi', 'w')
f_lo1 = open('../data/rand.lo', 'w')
f_qu1 = open('../data/rand.qu', 'w')

for i in range(len(num)):
    f_lo1.write(Flo[num[i]])
    f_fi1.write(Ffi[num[i]])
    f_qu1.write(Fqu[num[i]])

f_fi1.close()
f_qu1.close()
f_lo1.close()

f_fi_train = open('../data/rand_train.fi', 'w')
f_lo_train = open('../data/rand_train.lo', 'w')
f_qu_train = open('../data/rand_train.qu', 'w')
f_fi_dev = open('../data/rand_dev.fi', 'w')
f_lo_dev = open('../data/rand_dev.lo', 'w')
f_qu_dev = open('../data/rand_dev.qu', 'w')
f_fi_test = open('../data/rand_test.fi', 'w')
f_lo_test = open('../data/rand_test.lo', 'w')
f_qu_test = open('../data/rand_test.qu', 'w')

for i in range(len(num)):
    if i < 8000:
        f_lo_train.write(Flo[num[i]])
        f_fi_train.write(Ffi[num[i]])
        f_qu_train.write(Fqu[num[i]])
        continue
    if i < 10000:
        f_lo_dev.write(Flo[num[i]])
        f_fi_dev.write(Ffi[num[i]])
        f_qu_dev.write(Fqu[num[i]])
        continue
    f_lo_test.write(Flo[num[i]])
    f_fi_test.write(Ffi[num[i]])
    f_qu_test.write(Fqu[num[i]])


f_fi_train.close()
f_qu_train.close()
f_lo_train.close()
f_fi_dev.close()
f_qu_dev.close()
f_lo_dev.close()
f_fi_test.close()
f_qu_test.close()
f_lo_test.close()

In [27]:
import math
import os
import random
import sys
import time
import re

import numpy as np
import tensorflow as tf
import data_utils_tag

#_buckets = [(10, 8), (14, 12), (18, 15), (23, 20)]  
# _buckets = [(11, 8), (15, 12), (20, 16), (24, 21)]
_buckets = [(10, 8), (15, 12), (19, 16)]
max_size = None
data_set = [[] for _ in _buckets]
stat = dict()

with tf.gfile.GFile('../data/rand_train.qu', mode="r") as source_file:
    with tf.gfile.GFile('../data/rand_train.lo', mode="r") as target_file:
        with tf.gfile.GFile('../data/rand_train.ta', mode="r") as tag_file:
            source, target, tag = source_file.readline(), target_file.readline(), tag_file.readline()
            counter = 0
            while source and target and tag and (not max_size or counter < max_size):
                counter += 1
                if counter % 200 == 0:
                    print("  reading data line %d" % counter)
                    sys.stdout.flush()
                source_ids = [x for x in source.split()]
                target_ids = [x for x in target.split()]
                tag_ids = [x for x in tag.split()]
                target_ids.append(data_utils_tag.EOS_ID)
                if (len(source_ids), len(target_ids)) not in stat:
                    stat[(len(source_ids), len(target_ids))] = 0
                stat[(len(source_ids), len(target_ids))] += 1
                for bucket_id, (source_size, target_size) in enumerate(_buckets):
                    if len(source_ids) < source_size and len(target_ids) < target_size:
                        data_set[bucket_id].append([source_ids, tag_ids, target_ids])
                        break
                source, target, tag = source_file.readline(), target_file.readline(), tag_file.readline()

print "bucket 0: %d" % len(data_set[0])
print "bucket 1: %d" % len(data_set[1])
print "bucket 2: %d" % len(data_set[2])
#print "bucket 3: %d" % len(data_set[3])

for (s_size, t_size) in sorted(stat.keys()):
    print '(%d, %d): %d' %(s_size, t_size, stat[(s_size, t_size)])

  reading data line 200
  reading data line 400
  reading data line 600
  reading data line 800
  reading data line 1000
  reading data line 1200
  reading data line 1400
  reading data line 1600
  reading data line 1800
  reading data line 2000
  reading data line 2200
  reading data line 2400
  reading data line 2600
  reading data line 2800
bucket 0: 834
bucket 1: 752
bucket 2: 1214
(4, 5): 5
(4, 7): 28
(5, 5): 23
(5, 7): 39
(6, 5): 19
(6, 7): 250
(7, 3): 1
(7, 5): 7
(7, 7): 83
(7, 11): 5
(8, 5): 14
(8, 7): 54
(8, 9): 63
(9, 5): 33
(9, 7): 278
(9, 9): 27
(9, 11): 9
(9, 12): 74
(10, 5): 6
(10, 7): 95
(10, 9): 284
(10, 11): 28
(11, 5): 3
(11, 7): 55
(11, 8): 1
(11, 9): 72
(11, 11): 33
(11, 12): 216
(12, 7): 2
(12, 11): 43
(12, 13): 27
(13, 5): 1
(13, 7): 6
(13, 12): 155
(13, 13): 74
(14, 11): 19
(14, 12): 21
(14, 13): 15
(15, 11): 334
(15, 12): 42
(16, 8): 2
(17, 11): 10
(17, 13): 235
(18, 11): 9
